### Dreambooth 模型微调
DreamBooth 是一种深度学习生成模型，用于微调现有的文本到图像模型，由 Google Research 和波士顿大学的研究人员于 2022 年开发。最初使用 Google 自己的 Imagen 文本到图像模型开发，DreamBooth 的实现可以应用到其他文本到图像模型，它可以让模型在对一个主题的三到五张图像进行训练后生成更精细和个性化的输出。

接下来我们将使用 DreamBooth 来微调我们的 stable diffusion 模型.

#### Notebook 步骤
1. 导入 boto3, sagemaker python SDK
2. 构建 dreambooth fine-tuning 镜像
3. 实现模型微调
   * 配置超参
   * 创建训练任务
4. 测试

#### 1. 导入 boto3, sagemaker python SDK

In [4]:
import sagemaker
import boto3
from sagemaker.pytorch import PyTorch
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()
account_id = boto3.client('sts').get_caller_identity().get('Account')
region_name = boto3.session.Session().region_name

images_s3uri = 's3://{0}/dreambooth/images/'.format(bucket)
models_s3uri = 's3://{0}/stable-diffusion/models/'.format(bucket)
dreambooth_s3uri = 's3://{0}/stable-diffusion/dreambooth/'.format(bucket)

#### 2. 构建 dreambooth fine-tuning 镜像
  如果你使用较小的实例，如 ml.t3.xlarge，这一步将需要 60~90 分钟的时间

In [5]:
!./build_push.sh

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
Cloning into 'sd_dreambooth_extension'...
remote: Enumerating objects: 1926, done.
remote: Counting objects: 100% (1926/1926), done.
remote: Compressing objects: 100% (698/698), done.
remote: Total 1926 (delta 1248), reused 1834 (delta 1163), pack-reused 0
Receiving objects: 100% (1926/1926), 10.12 MiB | 21.91 MiB/s, done.
Resolving deltas: 100% (1248/1248), done.
Sending build context to Docker daemon   35.9MB
Step 1/19 : FROM ghcr.io/allenai/pytorch:1.12.1-cuda11.3-python3.9-v1.2.0
 ---> 56910f81bfd4
Step 2/19 : ENV PATH="/opt/ml/code:${PATH}"
 ---> Using cache
 ---> f07ea4da8ec5
Step 3/19 : ENV DEBIAN_FRONTEND noninteractive
 ---> Using cache
 ---> 670419a97e13
Step 4/19 : 

Submodule 'third_party/cutlass' (https://github.com/NVIDIA/cutlass.git) registered for path 'third_party/cutlass'
Submodule 'third_party/flash-attention' (https://github.com/HazyResearch/flash-attention.git) registered for path 'third_party/flash-attention'
Cloning into '/opt/ml/code/repositories/xformers/third_party/cutlass'...
Cloning into '/opt/ml/code/repositories/xformers/third_party/flash-attention'...
Submodule path 'third_party/cutlass': checked out '06eb90cc0daae633b1e25e80ace1ef81ac158baa'
Submodule path 'third_party/flash-attention': checked out 'a84d07283c23d5afa10fece6927da088f7fff81e'
Submodule 'csrc/flash_attn/cutlass' (https://github.com/NVIDIA/cutlass.git) registered for path 'third_party/flash-attention/csrc/flash_attn/cutlass'
Cloning into '/opt/ml/code/repositories/xformers/third_party/flash-attention/csrc/flash_attn/cutlass'...
Submodule path 'third_party/flash-attention/csrc/flash_attn/cutlass': checked out '319a389f42b776fae5701afcb943fc03be5b5c25'
Obtaining file

#### 3. 模型微调

   * image_uri: ecr仓库中的 docker 镜像地址
   * instance_type: 用于训练任务的实例大小 , 建议使用 ml.g4dn.xlarge, ml.g5.xlarge
   * class_prompt: 提示词类别
   * instance_prompt: 用于你的图片的关键词
   * model_name: 预训练的模型名称
   

In [ ]:
import json
def json_encode_hyperparameters(hyperparameters):
    for (k, v) in hyperparameters.items():
        print(k, v)
    
    return {k: json.dumps(v) for (k, v) in hyperparameters.items()}




image_uri = f'{account_id}.dkr.ecr.{region_name}.amazonaws.com/sd-dreambooth-finetuning'
instance_type = 'ml.g4dn.2xlarge'

instance_prompt="photo\ of\ aabbcc\  woman"
class_prompt="photo\ of\ a\ woman"
s3_model_output_location='s3://{}/{}/{}'.format(bucket, 'dreambooth', 'trained_models')
model_name="runwayml/stable-diffusion-v1-5"
#model_name="stabilityai/stable-diffusion-2"
instance_dir="/opt/ml/input/data/images/"
class_dir="/opt/ml/input/data/class_images/"



environment = {
    'PYTORCH_CUDA_ALLOC_CONF':'max_split_size_mb:32',
    'LD_LIBRARY_PATH':"${LD_LIBRARY_PATH}:/opt/conda/lib/"
}

hyperparameters = {
                    'model_name':'aws-trained-dreambooth-model',
                    'mixed_precision':'fp16',
                    'pretrained_model_name_or_path': model_name, 
                    'instance_data_dir':instance_dir,
                    'class_data_dir':class_dir,
                    'with_prior_preservation':True,
                    'models_path': '/opt/ml/model/',
                    'instance_prompt': instance_prompt, 
                    'class_prompt':class_prompt,
                    'resolution':512,
                    'train_batch_size':1,
                    'sample_batch_size': 1,
                    'gradient_accumulation_steps':1,
                    'learning_rate':5e-06,
                    'lr_scheduler':'constant',
                    'lr_warmup_steps':0,
                    'num_class_images':50,
                    'max_train_steps':300,
                    'save_steps':300,
                    'attention':'xformers',
                    'prior_loss_weight': 0.5,
                    'use_ema':True,
                    'train_text_encoder':False,
                    'not_cache_latents':True,
                    'gradient_checkpointing':True,
                    'save_use_epochs': False,
                    'use_8bit_adam': False
}

hyperparameters = json_encode_hyperparameters(hyperparameters)



   * 创建训练任务

In [ ]:
from sagemaker.estimator import Estimator
inputs = {
    'images': f"s3://sagemaker-{region_name}-{account_id}/dreambooth/images/"
}


estimator = Estimator(
    role = role,
    instance_count=1,
    instance_type = instance_type,
    image_uri = image_uri,
    hyperparameters = hyperparameters,
    environment = environment
)
estimator.fit(inputs)

In [28]:
dreambooth_model_data = estimator.model_data
print("Model artifact saved at:\n", dreambooth_model_data)

Model artifact saved at:
 s3://sagemaker-ap-southeast-1-687912291502/dreambooth-finetuning-v3-with-webui-2023-01-27-13-04-28-401/output/model.tar.gz


In [ ]:
!aws s3 ls s3://sagemaker-ap-southeast-1-687912291502/stable-diffusion/dreambooth/aws-db-new-model/working/unet/

#### 4. 测试
  现在你可以使用推理笔记本加载您训练的模型